### Images pre-processing => OK code Ready

In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import os

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#file paths
test_path = '/Users/sam/Desktop/X/ens100/data_large_files/input_test'
padim_trained = '/Users/sam/Desktop/X/ens100/data_large_files/PADIM.pkl'
test_pictures = '/Users/sam/Desktop/X/ens100/data_large_files/input_train'
test_labels = '/Users/sam/Desktop/X/ens100/data_large_files/Y_train_eVW9jym.csv'

classifier_file = '/Users/sam/Desktop/X/ens100/data_large_files/Classifier.pt'
image_folder = '/Users/sam/Desktop/X/ens100/data_large_files/input_test'



In [3]:
def rotate_and_crop_image(path, angle, crop_box, plot=False):
    """
    Applique une rotation d'un angle donné et un crop sur une image, puis enregistre le résultat en .png.
    Optionnellement, affiche l'image si plot=True.
    
    Arguments :
    - input_image_path : Chemin de l'image d'entrée.
    - output_image_path : Chemin pour enregistrer l'image modifiée.
    - angle : L'angle de rotation en degrés (positif dans le sens antihoraire).
    - crop_box : Un tuple (left, upper, right, lower) définissant les points du crop en pixels.
    - plot : Booléen, si True affiche l'image modifiée.
    """
    # Ouvrir l'image
    image = Image.open(path)
    
    # Appliquer la rotation (expand=True permet d'agrandir l'image pour qu'elle s'ajuste au cadre après rotation)
    rotated_image = image.rotate(angle, expand=True)
    
    # Appliquer le crop avec les points fournis
    cropped_image = rotated_image.crop(crop_box)
    
    # Enregistrer l'image au format PNG
    cropped_image.save(path, format='PNG')
    #print(f"Image enregistrée avec succès sous {path}")
    
    # Si plot=True, afficher l'image modifiée
    if plot:
        plt.imshow(cropped_image)
        plt.axis('off')  # Masquer les axes
        plt.title(f"Image après rotation de {angle}° et crop")
        plt.show()

In [4]:
# Dict defining rotation and cropping depending on component
rot_crop_data = {
        "Die01":[55,(340, 120, 500, 680)], # (left, upper, right, lower)
        "Die02":[-44, (480, 210, 640, 930)],
        "Die03":[134, (460, 200, 620, 920)],
        "Die04":[35, (310, 130, 470, 690)]
}

In [5]:

# Pre-process test data

test_df = pd.read_csv(test_labels)

#for index, row in test_df.iterrows():
#    path = os.path.join(test_pictures, row['filename'])
#    rotate_and_crop_image(path, rot_crop_data[row['lib']][0], rot_crop_data[row['lib']][1], plot=False)

### Anomaly detection (PADIM) => OK

In [6]:
import random
from random import sample
import argparse
import numpy as np
import os
import pickle
from tqdm import tqdm
from collections import OrderedDict
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import mahalanobis
from scipy.ndimage import gaussian_filter
from skimage import morphology
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import matplotlib

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.models import wide_resnet50_2, resnet18

#from Users.sam.Desktop.X.ens100.data_large_files import Classifier

import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        self.images = []


        for img_name in os.listdir(self.data_path):
            img_path = os.path.join(self.data_path, img_name)
            self.images.append(img_path)


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")  # Charger l'image
    
        # Appliquer les transformations
        if self.transform:
            image = self.transform(image)  # Appliquer les transformations à l'image

        filename = os.path.basename(img_path)

        return filename, image                               


def denormalization(x):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    x = (((x.transpose(1, 2, 0) * std) + mean) * 255.).astype(np.uint8)
    
    return x


def embedding_concat(x, y):
    B, C1, H1, W1 = x.size()
    _, C2, H2, W2 = y.size()
    s = int(H1 / H2)
    x = F.unfold(x, kernel_size=s, dilation=1, stride=s)
    x = x.view(B, C1, -1, H2, W2)
    z = torch.zeros(B, C1 + C2, x.size(2), H2, W2)
    for i in range(x.size(2)):
        z[:, :, i, :, :] = torch.cat((x[:, :, i, :, :], y), 1)
    z = z.view(B, -1, H2 * W2)
    z = F.fold(z, kernel_size=s, output_size=(H1, W1), stride=s)

    return z


# Charger les données d'entraînement
with open(padim_trained, 'rb') as f:
    train_outputs = pickle.load(f)

model = wide_resnet50_2(pretrained=True, progress=True)
t_d = 1792
d = 550


# device setup
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

model.to(device)
model.eval()
random.seed(1024)
torch.manual_seed(1024)
if use_cuda:
    torch.cuda.manual_seed_all(1024)

idx = torch.tensor(sample(range(0, t_d), d))

# set model's intermediate outputs
outputs = []

def hook(module, input, output):
    outputs.append(output)

model.layer1[-1].register_forward_hook(hook)
model.layer2[-1].register_forward_hook(hook)
model.layer3[-1].register_forward_hook(hook)


transform = transforms.Compose([
                                transforms.Resize((128, 128)),  # Redimensionner les images
                                transforms.ToTensor(),            # Convertir en tenseur
                                ])

test_dataset = CustomDataset(test_path, transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, pin_memory=True)
test_outputs = OrderedDict([('layer1', []), ('layer2', []), ('layer3', [])])

test_imgs = []
filenames_list = []
scores_list = []

# extract test set features
for filename, x in tqdm(test_dataloader, '| feature extraction | test |'):
    filenames_list.extend(filename)  # Collect filenames for the batch
    test_imgs.extend(x.cpu().detach().numpy())
    # model prediction
    with torch.no_grad():
        _ = model(x.to(device))
    # get intermediate layer outputs
    for k, v in zip(test_outputs.keys(), outputs):
        test_outputs[k].append(v.cpu().detach())
    # initialize hook outputs
    outputs = []
for k, v in test_outputs.items():
    test_outputs[k] = torch.cat(v, 0)

# Embedding concat
embedding_vectors = test_outputs['layer1']
for layer_name in ['layer2', 'layer3']:
    embedding_vectors = embedding_concat(embedding_vectors, test_outputs[layer_name])

# randomly select d dimension
embedding_vectors = torch.index_select(embedding_vectors, 1, idx)

# calculate distance matrix
B, C, H, W = embedding_vectors.size()
embedding_vectors = embedding_vectors.view(B, C, H * W).numpy()
dist_list = []
for i in range(H * W):
    mean = train_outputs[0][:, i]
    conv_inv = np.linalg.inv(train_outputs[1][:, :, i])
    dist = [mahalanobis(sample[:, i], mean, conv_inv) for sample in embedding_vectors]
    dist_list.append(dist)

dist_list = np.array(dist_list).transpose(1, 0).reshape(B, H, W)

# upsample
dist_list = torch.tensor(dist_list)
score_map = F.interpolate(dist_list.unsqueeze(1), size=x.size(2), mode='bilinear',
                            align_corners=False).squeeze().numpy()

# apply gaussian smoothing on the score map
for i in range(score_map.shape[0]):
    score_map[i] = gaussian_filter(score_map[i], sigma=4)

# Normalization
max_score = score_map.max()
min_score = score_map.min()
scores = (score_map - min_score) / (max_score - min_score)

# calculate image-level ROC AUC score
img_scores = scores.reshape(scores.shape[0], -1).max(axis=1)

# Store scores and filenames
for filename, score in zip(filenames_list, img_scores):
    scores_list.append((filename, score))

/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet50_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet50_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
| feature extraction | test |:  36%|███▋      | 12/33 [01:08<02:14,  6.41s/it]

In [6]:
print(scores_list)

[('06c5f2fcc05710bdd24a660307f1f8aec53d38fe2258d97b36e7266e285e838f.png', 0.24338196), ('3233bb04a275664c0415cfd1ca84d721f6ee56e09595c8cc6ac0673b8711f3d0.png', 0.5059239), ('cb2ee07e3969eb12a3149395f0034f7c285d4aa6d9f6fb38fbc83ae9ab9a6499.png', 0.40772247), ('91eda638e6a3abe3b3c653af492177ffa2133b4f162262c196f1822c6a22ff8a.png', 0.15159838), ('67406bc029971e414e61363c6f0997a1477c73480f2ea32171b1c4b700007dc1.png', 0.16498451), ('b5e89885021e7e45129af5e2a43edd5938cf6160d1f3a603b888da5b75aa8d73.png', 0.21432307), ('3944ed158b9eea08cae20a150c5b947e8e5574b088d3a2ddcdaa362b771418ed.png', 0.31562677), ('f6351ae439a817d52bdaca00bd91a1698f027f28bec556846bcc03fe147da5cf.png', 0.15781191), ('e4cb8265a658bd78a533957ec2c402a2ebbeeeee64b27dec11ceb7f5465213cf.png', 0.2317898), ('13be8686440a9e8f62d42383c30758bf06b6e2ec4fedd61dfc92d43a94bfed7b.png', 0.37727642), ('fbc82e5e9e49f9c288071a617decbb7c36d67abcb823f4a3f9ea93b204169026.png', 0.22814181), ('214faf9942c9c6801df7bc5f0f5775a680645b9afc8f8d58aef24

In [7]:
y_pred_anomaly = scores_list

NameError: name 'scores_list' is not defined

### Classifier    => OK code Ready

In [8]:
# Création de dataset utilisé par le Classifier
class ImageDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_folder, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, self.image_files[idx]  # Renvoie l'image et son nom

In [9]:


import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

# Transformation des images pour l'entrée du modèle
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
])

# Chargement du dataset
dataset = ImageDataset(image_folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# device setup
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

# Load classifier model
model = torch.load(classifier_file)
model.to(device)
model.eval()  

# Passer les images dans le modèle
y_pred_classifier = []
filenames_list = []
output_classifier=[]


with torch.no_grad():  # Désactive le calcul des gradients
    for images, filenames in dataloader:
        images = images.to(device)
        outputs = model(images)
        # Move the outputs to CPU and convert to numpy
        output_classifier.extend(outputs.cpu().numpy())
        filenames_list.extend(filenames)  # Ajouter les noms de fichiers

# Store scores and filenames
for filename, score in zip(filenames_list, output_classifier):
    y_pred_classifier.append((filename, score))

print(y_pred_classifier)

[('06c5f2fcc05710bdd24a660307f1f8aec53d38fe2258d97b36e7266e285e838f.png', array([2.5547522e-05, 6.2929641e-05, 7.5392156e-05, 3.3242140e-05,
       9.9974924e-01, 5.3632826e-05], dtype=float32)), ('3233bb04a275664c0415cfd1ca84d721f6ee56e09595c8cc6ac0673b8711f3d0.png', array([9.9998176e-01, 1.0861713e-05, 2.0376174e-06, 3.0980834e-06,
       1.5077434e-07, 2.1771605e-06], dtype=float32)), ('cb2ee07e3969eb12a3149395f0034f7c285d4aa6d9f6fb38fbc83ae9ab9a6499.png', array([8.46295152e-05, 1.28601969e-04, 1.45054029e-04, 1.10985384e-04,
       9.99410272e-01, 1.20463839e-04], dtype=float32)), ('91eda638e6a3abe3b3c653af492177ffa2133b4f162262c196f1822c6a22ff8a.png', array([3.0849194e-05, 3.6405017e-05, 4.6369430e-05, 3.0936812e-05,
       9.9982196e-01, 3.3495424e-05], dtype=float32)), ('67406bc029971e414e61363c6f0997a1477c73480f2ea32171b1c4b700007dc1.png', array([4.2220527e-05, 6.0382161e-05, 4.9738450e-05, 3.9097016e-05,
       9.9977225e-01, 3.6396057e-05], dtype=float32)), ('b5e89885021e7e45

### Calcul de la performance

In [10]:
# Check classifier predictions lenght = anomaly detection prediction length

if len(y_pred_classifier) != len(y_pred_anomaly):
    print("Erreur: les deux prédictions n'ont pas la meme dimension")

In [24]:
# Convertir y_pred_anomaly en un dictionnaire
anomaly_dict = dict(y_pred_anomaly)

# Convertir y_pred_classifier en un dictionnaire
classifier_dict = dict(y_pred_classifier)

# Fusionner les deux dictionnaires basés sur le filename
# Assurer que tous les filenames sont présents dans les deux modèles
merged_data = []
for filename in anomaly_dict.keys():
    if filename in classifier_dict:
        anomaly_score = anomaly_dict[filename]
        classifier_probs = classifier_dict[filename]
        merged_data.append((filename, anomaly_score, *classifier_probs))  # Fusionner les données

# Créer un DataFrame pandas avec les résultats
final_df = pd.DataFrame(merged_data, columns=['filename', 'p_drift', 'p0', 'p1', 'p2', 'p3', 'p4', 'p5'])


In [34]:
final_df.head()

,filename,p_drift,p0,p1,p2,p3,p4,p5,window,lib,Label,y_pred
0,06c5f2fcc05710bdd24a660307f1f8aec53d38fe2258d9...,0.243382,0.000026,0.000063,0.000075,0.000033,9.997492e-01,0.000054,2003,Die04,4,4
1,3233bb04a275664c0415cfd1ca84d721f6ee56e09595c8...,0.505924,0.999982,0.000011,0.000002,0.000003,1.507743e-07,0.000002,2003,Die03,6,6
2,cb2ee07e3969eb12a3149395f0034f7c285d4aa6d9f6fb...,0.407722,0.000085,0.000129,0.000145,0.000111,9.994103e-01,0.000120,2003,Die01,4,4
3,91eda638e6a3abe3b3c653af492177ffa2133b4f162262...,0.151598,0.000031,0.000036,0.000046,0.000031,9.998220e-01,0.000033,2005,Die04,4,4
4,67406bc029971e414e61363c6f0997a1477c73480f2ea3...,0.164985,0.000042,0.000060,0.000050,0.000039,9.997723e-01,0.000036,2005,Die01,4,4


In [20]:
#### Get the y_test

test_df = pd.read_csv(test_labels)

classes_dict = {
    "GOOD" : 0,
    "Boucle plate" : 1,
    "Lift-off blanc" : 2,
    "Lift-off noir" : 3,
    "Missing" : 4,
    "Short circuit MOS" : 5,
    "Drift" : 6
}

# Replace the labels by the corresponding vlaues in the dict

test_df['Label'] = test_df['Label'].replace(classes_dict)

# Check the new values
test_df['Label'].unique()


array([4, 0, 6, 2, 3, 5, 1])

In [29]:
# merge the results and true results DFs

merged_df = pd.merge(final_df, test_df, on='filename', how='inner')

In [30]:
print(len(final_df))
print(len(test_df))
print(len(merged_df))

1055
1055
1055


In [33]:
import numpy as np

def define_classe(row, threshold=0.5):
    # Accéder à la valeur de p_drift dans la ligne
    p_drift = row['p_drift']
    
    # Extraire les probabilités des classes (colonnes 2 à 7)
    p_classes = row[['p0', 'p1', 'p2', 'p3', 'p4', 'p5']].values
    
    # Si le drift est supérieur au seuil, retourner 6, sinon index du max des classes
    if p_drift > threshold:
        return 6
    else:
        return p_classes.argmax()  # Utiliser argmax pour obtenir l'indice du max

# Appliquer la fonction à chaque ligne du DataFrame
merged_df['y_pred'] = merged_df.apply(define_classe, axis=1)


In [37]:
import numpy as np

def penalty_weighted_accuracy(y_true, y_pred, penalty_matrix):
    total_penalty = 0
    max_penalty = penalty_matrix.max()  # Maximum possible penalty in the matrix
    n = len(y_true)  # Total number of samples

    for i in range(n):
        true_class = y_true[i]
        predicted_class = y_pred[i]
        
        # If misclassified, add the penalty; if correct, penalty is 0
        if true_class != predicted_class:
            total_penalty += penalty_matrix[true_class, predicted_class]
    
    # Normalize penalty by the worst-case penalty (n * max_penalty)
    normalized_penalty = total_penalty / (n * max_penalty)
    
    # Calculate PWA
    pwa = 1 - normalized_penalty
    return pwa

# Define the penalty matrix based on the design example
penalty_matrix = np.array([
    [0, 100, 100, 100, 100, 100, 10000],    # Good
    [10000, 0, 1, 1, 1, 1, 1000],           # Defect1
    [10000, 1, 0, 1, 1, 1, 1000],           # Defect2
    [10000, 1, 1, 0, 1, 1, 1000],           # Defect3
    [10000, 1, 1, 1, 0, 1, 1000],           # Defect4
    [10000, 1, 1, 1, 1, 0, 1000],           # Defect5
    [10000, 1000, 1000, 1000, 1000, 1000, 0] # Drift
])

# Calculate Penalty-Weighted Accuracy
pwa = penalty_weighted_accuracy(merged_df['Label'], merged_df['y_pred'], penalty_matrix)
print(f"Penalty-Weighted Accuracy: {pwa}")

Penalty-Weighted Accuracy: 0.9755256872037915
